# 0. Set-Ups

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import pyspark

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[3]") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")
#to not cache datafrimes... this may not create repeatable results
spark.conf.set("spark.databricks.io.cache.enabled", "false")

In [4]:
d = [
    {"a":"a", "b": 1},
    {"a":"b", "b": 2},
    {"a":"c", "b": 3},
    {"a":"d", "b": 4},
    {"a":"e", "b": 5},
    {"a":"e", "b": 6},
    {"a":"f", "b": 7},
    {"a":"g", "b": 8},
    {"a":"h", "b": 9},
    {"a":"i", "b": 10},
    {"a":"j", "b": 11},
    {"a":"k", "b": 12},
    {"a":"a", "b": 13},
    {"a":"b", "b": 13},
]
ddl_schema = "a string, b int"
sdf = spark.createDataFrame(d, schema=ddl_schema)

# 1. Lazy Execution and actions

In [5]:
sdf_lazy = sdf.filter(f.col("b") > 5)

In [6]:
sc.setJobDescription("LazyExecution")
sdf_lazy.count()

9

In [7]:
sdf_lazy.rdd.getNumPartitions()

3

# 5. Wide transformation

In [8]:
sc.setJobDescription("Wide2")
sdf_w = sdf.groupBy("a").count()
sdf_w.write.format("noop").mode("overwrite").save()

In [9]:
sdf_w.rdd.getNumPartitions()

200